In [77]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split, feature_dataload
from feature_engineering import *
from candidate import *
from tqdm import tqdm

In [78]:
import os
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
    # When Running on the CuDNN backend, two further options muset be set
#     torch.backends.cudnn.deteministic =True
#     torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

# dataload
- 편의상 df_train_week 변수 통일하여 사용함

In [79]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [80]:
# # # 전체 데이터 이용시
# df_history = pd.read_csv(path+'history_data.csv')

# # ## 날짜 전처리
# df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
# df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
# df_history = df_history.drop("log_time", axis=1)

# df_train_week = df_history.copy()

# # # month 이용
# # # df_train_week = df_train_month.copy()

# feature dataload

In [81]:
# data load
watch_df, buy_df, search_df, meta_df, profile_df = feature_dataload(path)

../../src/dataload.py:87: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path + "meta_data.csv")


In [82]:
# feature engineering
# df_train_week = history_feature_engineering(df_train_week)

# short_con_freq_feature = df_train_week[["album_id","short_trailer","continuous_play","album_viewcount_freq"]]

watch_df_feature = watch_feature_engineering(watch_df)
watch_df_feature = watch_df_feature.drop_duplicates(subset=["profile_id","album_id"])

paid_df_feature = paid_feature_engineering(df_train_week, buy_df)

searched_df_feature = searched_feature_engineering(df_train_week, search_df)

meta_df_feature = meta_feature_engineering(meta_df)
meta_df_feature = meta_df_feature.drop(columns="sub_title").drop_duplicates()

profile_df_feature = profile_feature_engineering(profile_df)
profile_df_feature = profile_df_feature[["profile_id","sex","age","age_bin"]]

# day_week feature engineering
- to split label, train data

In [83]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-23 00:00:00
min day: 0 max day: 53
min week: 0 max week: 7


# label & train split

In [84]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week)

split last week: 7


In [85]:
n = 25

In [86]:
len(df_train), len(df_train_week)

(945518, 945518)

# prerpocess

In [87]:
# 총 3개 파일 사용

# log dt 기준 duplicates
df_train_week = df_train.drop_duplicates(subset=["profile_id","album_id","log_dt"])
# ss_id 기준 duplicates
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
# album_id, profile_id 기준으로 duplicates
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

# 전체 유저 한정
customers = df_train.profile_id.unique()
# 라벨 한정
# customers = label_df.profile_id.unique()

# day week features

In [88]:
interaction_day_week_first = day_week_feature(df_train_week)
hour_feature = hour_feature(df_train_week)

../../src/feature_engineering.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)


# candidate generation

# MP

In [89]:
MP_cand = most_popular(df_train)

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [90]:
%%time
general_MP_cand, general_MP_feature = general_most_popular(df_train)

CPU times: user 50.3 ms, sys: 11.5 ms, total: 61.8 ms
Wall time: 60.9 ms


# personal_MP

In [91]:
%%time
personal_MP_candidate, personal_MP_feature = personal_most_popular(personal_train)

100%|██████████████████████████████████████| 8057/8057 [00:08<00:00, 957.73it/s]


CPU times: user 8.73 s, sys: 112 ms, total: 8.85 s
Wall time: 8.84 s


# MP_user_genre

In [92]:
# 이전에는 meta 중복때문에 장르 count가 잘못 뻥튀기 되어서 잘못 MP가 매겨짐
# %%time
genre_candidate = user_genre_most_popular(df_train, meta_df)

# ALS MF candidate

In [93]:
%%time
from ALS_MF import MF
model = MF(df_train_week, clf={'factors': 200, 'regularization': 0.005, 'iterations': 3, 'n': 100})
als_candidate, item_factors_feature, user_factors_feature = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20383, 200) user vector shape (8057, 200)
CPU times: user 19.6 s, sys: 17.9 s, total: 37.5 s
Wall time: 20.9 s


# apriori candidaate

In [94]:
%%time
from apriori import apriori_train, apriori_candidate
rules_confidence_item_week = apriori_train(df_train, 0.1, 0.8)
apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

CPU times: user 3.64 s, sys: 65.1 ms, total: 3.71 s
Wall time: 3.7 s


# candidate merge

In [95]:
general_MP_cand.album_id.nunique(), personal_MP_candidate.album_id.nunique(), genre_candidate.album_id.nunique(),\
als_candidate.album_id.nunique(), apriori_candidate.album_id.nunique()

(17, 2148, 150, 8448, 4)

In [96]:
candidate_0 = MP_cand
candidate_1 = general_MP_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
candidate_5 = apriori_candidate.copy()

cand = pd.concat([candidate_0, candidate_1, candidate_2, candidate_3, candidate_4, candidate_5])
cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.album_id.nunique())

cand 데이터 수: 1204936
cand nunique:  8473


# features merge

In [97]:
# general counts feature merge
candidate = pd.merge(cand, general_MP_feature, how='left', on='album_id')
# personal_count feature merge
candidate = pd.merge(candidate, personal_MP_feature, how='left', on=['profile_id','album_id'])
# # apriori feature merge
candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [98]:
print('cand 데이터 수:',len(candidate))

cand 데이터 수: 1204936


In [99]:
# meta feature merge

'''
아직 사용 안하고 있음
# profile feature add
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
# meta feature add
candidate_add_features = pd.merge(candidate_add_features, meta_df.drop_duplicates('album_id'), how='left', on='album_id')

# catetegory feature label encoding

columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
'genre_mid','cast_1','cast_2','cast_3']
from sklearn.preprocessing import LabelEncoder
for col in columns:
    LE = LabelEncoder()
    candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

# drop columns

candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
                                                              'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
                                                              ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
                                                              ,'ch_interest_keyword_cd_3',\
                                                                'genre_mid','cast_1','cast_2','cast_3'])
                                                                
'''
candidate_add_features = candidate.copy()
# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

# interaction day, week merge
candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_first, how="left",on=["profile_id","album_id"])

# hour feature merge
# candidate_add_features = pd.merge(candidate_add_features, hour_feature, how="left",on=["profile_id","album_id"])

In [102]:
# short_con_freq_feature = short_con_freq_feature[["album_id","album_viewcount_freq"]].drop_duplicates()
# candidate_add_features = pd.merge(candidate_add_features,short_con_freq_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features,watch_df_feature, how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, meta_df_feature, how="left",on="album_id")
candidate_add_features = pd.merge(candidate_add_features, profile_df_feature, how="left",on="profile_id")
# candidate_add_features = pd.merge(candidate_add_features,searched_df_feature,how="left",on=["profile_id","album_id"])
# ->> duplicates 문제 발생
candidate_add_features = pd.merge(candidate_add_features,paid_df_feature,how="left",on=["profile_id","album_id"])

# 타입오류 픽처 제거
candidate_add_features = candidate_add_features.drop(columns=["onair_date","title", "genre_small", "cast_1", "cast_2", "cast_3", "cast_4", "cast_5", "cast_6", "cast_7"])

In [103]:
len(candidate_add_features)

1204936

In [104]:
# 라벨 부여
train_df = pd.merge(candidate_add_features, label_df, how='left', on=['profile_id','album_id'])
train_df['rating'] = train_df['rating'].fillna(0)

# 비복원 추출 defalut
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# model

In [105]:
# feature selection 컬럼만 학습
# train_df = train_df[['profile_id','album_id','day', 'hour_14', 'personal_counts', 'week', 'total_counts',
#        'item_vector66', 'user_vector51', 'user_vector119', 'item_vector76',
#        'user_vector59', 'item_vector97', 'user_vector164', 'user_vector196',
#        'user_vector178', 'user_vector147', 'user_vector99', 'user_vector167',
#        'item_vector165', 'user_vector75', 'user_vector50', 'user_vector136',
#        'user_vector108', 'user_vector131', 'user_vector18', 'user_vector49',
#        'user_vector62', 'user_vector152', 'user_vector177', 'item_vector119','rating']]

In [106]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
X_train, sample_sumbission = lgbmrank.valid_evaluation()

                 feature_importances
total_counts             1329.509195
general_counts              0.000000
personal_counts        378088.076520
support                     0.000000
confidence                  0.000000
...                              ...
run_time                    0.000000
sex                         0.000000
age                         0.000000
age_bin                     0.000000
paid_label                  0.000000

[418 rows x 1 columns]
week performance
lgbm ndcg: 0.19806884782703654


In [ ]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [ ]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [ ]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [ ]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [ ]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

In [ ]:
sumbission_cold

In [ ]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [ ]:
# sumbission_cold.to_csv('lgbm_candidate_submission_feature_add_ver8.csv', index=False)